<a href="https://colab.research.google.com/github/wuyun1/BlankCordovaApp1/blob/master/GPT2_with_JS_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2 that  runs from colab with Javascript interface
this code is a slightly modified version of [this](https://github.com/gpt2ent/gpt2colab-js)


**STEPS:**

1. Runtime -> Change runtime type -> Hardware accelerator -> GPU
2. Runtime -> Reset all runtimes
3. Runtime -> Run all
4. Scroll down and wait until you see the little window
5. Type text
6. The button *Generate with GPT-2* will invoke GPT-2 and it will continue your text.



In [1]:
%tensorflow_version 1.x
!git clone https://github.com/gpt2ent/gpt-2-simple.git
%cd gpt-2-simple
!git checkout context-trim
!pip install .
%cd ..
!git clone https://github.com/gpt2ent/gpt2colab-js.git
%cd gpt2colab-js

TensorFlow 1.x selected.
Cloning into 'gpt-2-simple'...
remote: Enumerating objects: 568, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 568 (delta 9), reused 20 (delta 8), pack-reused 542
Receiving objects: 100% (568/568), 227.74 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (296/296), done.
/content/gpt-2-simple
Branch 'context-trim' set up to track remote branch 'context-trim' from 'origin'.
Switched to a new branch 'context-trim'
Processing /content/gpt-2-simple
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.2-py

In [2]:
import gpt_2_simple as gpt2

import os
import requests
import tensorflow as tf

import re

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
#Determining the graphics card used by colab: full model can run only on P100

try:
    !cat /proc/driver/nvidia/gpus/0000:00:04.0/information >> /content/card_info.txt
    with open('/content/card_info.txt','r') as f:
        graphics_card = re.split('\n|\t\t ',f.read())[1]
        print(graphics_card)

    if not graphics_card.startswith("Tesla P100"):
        print("="*90+'\n'+"="*90+'\n\n')
        print('\n\tYour current GPU - %s - cannot fit the full GPT-2 model!' % graphics_card)
        print('\n\tFalling back on 774M model.')
        print('\n\tNothing I can do. just pray to Google to give you a P100')
        print('\t\tnext time. ¯\_(ツ)_/¯')
        print('\n\tAlso you might try TPU runtime.')
        print('\n\n'+"="*90+'\n'+"="*90+'\n\n')
        model_name = "774M"
        spinner_speed = "300ms"
    else:
        print('GPU: %s' % graphics_card)
        model_name = "1558M"
        spinner_speed = '400ms'
except IndexError:
    print("="*90+'\n'+"="*90+'\n\n')
    print('\n\tYou\'re not in a GPU runtime.\n')
    print('\n\tTrying 1558M model anyways - assuming you\'re on a good TPU.')
    print('\n\tIf it fails, you have to go to Runtime -> Change runtime type')
    print('\n\tand choose GPU.')
    print('\n\n'+"="*90+'\n'+"="*90+'\n\n')
    model_name = "1558M"
    spinner_speed = "1200ms"


#Overwrite default model choice
#model_name = "1558M"
#model_name = "774M"
#model_name = "124M"
#model_name = "355M"


if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)
  
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name=model_name)

generate_count = 0

cat: '/proc/driver/nvidia/gpus/0000:00:04.0/information': No such file or directory



	You're not in a GPU runtime.


	Trying 1558M model anyways - assuming you're on a good TPU.

	If it fails, you have to go to Runtime -> Change runtime type

	and choose GPU.






Fetching checkpoint: 1.05Mit [00:00, 407Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.01Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 715Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 6.23Git [02:21, 44.0Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 51.7Mit/s]                                               
Fetching model.ckpt.meta: 2.10Mit [00:00, 9.00Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 6.63Mit/s]


Loading pretrained model models/1558M/model.ckpt
INFO:tensorflow:Restoring parameters from models/1558M/model.ckpt


In [4]:
import google.colab.output

def overlap(a, b):
    return max(i for i in range(len(b)+1) if a.endswith(b[:i]))


def ai_generate(prefix, temp, top_k, length):
    global sess
    global generate_count

    temp = float(temp)
    top_k = int(top_k)
    length = int(length)
    result = gpt2.generate(sess, model_name=model_name, prefix=prefix, temperature=temp,
                        top_k=top_k, length=length, include_prefix=False, return_as_list=True)[0]
    
    j = overlap(prefix, result)
    result = result[j:]
    
    generate_count += 1
    if generate_count == 6:
          #prevent memory leak as in https://github.com/minimaxir/gpt-2-simple/issues/71
          tf.reset_default_graph()
          sess.close()
          sess = gpt2.start_tf_sess()
          gpt2.load_gpt2(sess, model_name=model_name)
          generate_count = 0
    return result

#register callback for Javascript
google.colab.output.register_callback('ai_generate', ai_generate)

In [5]:
from IPython.display import HTML

#spinner from https://codepen.io/vovchisko/pen/vROoYQ
spinner_css = """
<style>
@keyframes c-inline-spinner-kf {
  0% {
    transform: rotate(0deg);
  }
  100% {
    transform: rotate(360deg);
  }
}

.c-inline-spinner,
.c-inline-spinner:before {
  display: inline-block;
  width: 11px;
  height: 11px;
  transform-origin: 50%;
  border: 2px solid transparent;
  border-color: #74a8d0 #74a8d0 transparent transparent;
  border-radius: 50%;
  content: "";
  animation: linear c-inline-spinner-kf """+spinner_speed+""" infinite;
  position: relative;
  vertical-align: inherit;
  line-height: inherit;
}
.c-inline-spinner {
  top: 3px;
  margin: 0 3px;
}
.c-inline-spinner:before {
  border-color: #74a8d0 #74a8d0 transparent transparent;
  position: absolute;
  left: -2px;
  top: -2px;
  border-style: solid;
}
</style>
"""

input_form = """
<link rel="stylesheet" href="https://unpkg.com/purecss@1.0.1/build/pure-min.css" integrity="sha384-oAOxQR6DkCoMliIh8yFnu25d7Eq/PHS21PClpwjOTeU2jRSq11vu66rf90/cZr47" crossorigin="anonymous">

<div style="background-color:white; border:solid #ccc; width:800px; padding:20px; color: black;">
<p>You have currently loaded <strong>GPT-2 %s</strong> model</p>
<textarea id="main_textarea" cols="75" rows="20" placeholder="Type something..." style="font-family: 'Liberation Serif', 'DejaVu Serif', Georgia, 'Times New Roman', Times, serif; font-size: 13pt; padding:10px;"></textarea><br>
<div class="pure-form pure-form-aligned">
    <div class="pure-control-group">
      <label for="temp"><strong>Temperature:</strong></label>
      <input type="number" min="0.00" max="999.99" step="0.01" id="temp" value="0.70" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="top_k"><strong>top_k:</strong></label>
        <input type="number" min="0" max="9999" id="top_k" value="40" style="background-color: white;">
    </div>
    <div class="pure-control-group">
        <label for="length"><strong>Generate how much:</strong></label>
        <input type="number" id="length" min="1" max="1023" value="10" style="background-color: white;">
    </div>
    <div style="width: 300px; display: block; margin-left: auto !important; margin-right: auto !important;">
        <p><button class="pure-button pure-button-primary" style="font-size: 125%%;" onclick="generate()">Generate with GPT-2</button>
        <span class="c-inline-spinner" style="visibility: hidden;" id="spinner"></span></p>
        <div id="timeElapsed"></div>
    </div>
</div>
</div>
""" % model_name

javascript = """
<script type="text/Javascript">

    var startTime, endTime;

      function start() {
        startTime = new Date();
      };

      function end() {
        endTime = new Date();
        var timeDiff = endTime - startTime; //in ms
        // strip the ms
        timeDiff /= 1000;

        // get seconds 
        var seconds = Math.round(timeDiff);
        return seconds;
      };

    function desanitize(text) {
        return text.slice(1,-1).replace(/\\\\n/g, "\\n").replace(/\\\\'/g, "'");
    };

    function add_text(text) {
        var deftext = document.getElementById('main_textarea').value;
        document.getElementById('main_textarea').value = deftext + text;
    };

    function generate(){
        var prefix = document.getElementById('main_textarea').value;
        var temp = document.getElementById('temp').value;
        var top_k = document.getElementById('top_k').value;
        var length = document.getElementById('length').value;
        
        var kernel = google.colab.kernel;

        start();
        var resultPromise = kernel.invokeFunction("ai_generate", [prefix,temp,top_k,length]); // developer, look here
        resultPromise.then(
            function(value) {
              add_text(desanitize(value.data["text/plain"]));
              document.getElementById('spinner').style = "visibility: hidden;";
              document.getElementById('timeElapsed').innerHTML = 'Inference time: ' +end() + ' seconds';
        });
        document.getElementById('spinner').style = "visibility: visible;";
    };

</script>
"""

HTML(spinner_css + input_form + javascript)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
